# Physics 494/594
## Mean Field Thery for the Ising Model

In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')
import ml4s

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s.set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
π = np.pi

## Last Time

### [Notebook Link: 26_Convolutional_Networks_Implementation.ipynb](./26_Convolutional_Networks_Implementation.ipynb)

- Convolutional network implementation in tensorflow
- Re-train on MNIST and explore improved generalizability

## Today
- Implement a numerical solution for the mean field magnetization of the 2D Ising model


### We identified the mean field solution for the magnetization of the 2D Ising model

\begin{equation}
\langle m \rangle = \tanh \left(\frac{ z J\langle m \rangle + h}{k_{\rm B}T}\right)
\end{equation}

As we are interested in the existince of a ferromagnetic phase in the absence of a magnetic field, set $h = 0$.

#### Let's begin by looking for a graphical solution.  

Define $x = zJ/(k_{\rm B} T)$: at fixed $J$, $x\gg 1$ corresponds to low temperature while $x\ll 1$ is high temperature.

We know that $\langle m \rangle$ is bounded between $-1 \le \langle m \rangle \le 1$.

In [ ]:
m = np.linspace(-1.1,1.1,1000)

In [ ]:
plt.plot(m,m,color='k', label=r'$y=\langle m \rangle $')
for cx in [10, 2,1.25,1.0,0.5]:
    plt.plot(m,np.tanh(m*cx), label=f'$x = {cx:4.2f}$')
plt.xlabel(r'$\langle m \rangle$')
plt.legend(loc='lower right')
plt.title('Mean Field Theory');

## Constructing the solution as a function of temperature

It appears that there is always at least one solution for $0 \le x \le \infty $ which corresponds to $\infty \ge T \ge 0$. 

In [ ]:
from scipy.optimize import fsolve

#### Find the solution of the equation

\begin{equation}
\langle m \rangle - \tanh \left(\langle m \rangle x\right)  = 0
\end{equation}

with 

\begin{equation}
x = \frac{zJ}{k_{\rm B} T}\, .
\end{equation}

As there is clear time reversal symmetry ($\langle m \rangle \to \langle -m \rangle$) we will only take the positive magnetization solution. 

In [ ]:
def mean_field_eqn(m,cx):
    '''The mean field equation for the magnetization.'''
    return m - np.tanh(m*cx)

In [ ]:
x = np.linspace(0.5,100,10000)
mx = np.array([fsolve(mean_field_eqn, 1.1, args=(cx)) for cx in x])

In [ ]:
plt.plot(1.0/x,mx, linewidth=4)
plt.xlabel(r'Temperature  $k_{\rm B} T / Jz$')
plt.ylabel(r'Magnetization  $\langle m \rangle$')
plt.text(0.15,0.5,'Ferromagnet', fontsize=16)
plt.text(1.2,0.5,'Paramagnet', fontsize=16)
plt.ylim(-0.001,1.05);
plt.xlim(0,2)

## Compare with the exact solution due to Onsager

\begin{equation}
m=\left[1-\left(\sinh \frac{2J}{k_{\rm B}T}\right)^{-4}\right]^{\frac {1}{8}}
\end{equation}

In [ ]:
def magnetization_exact_(T):
    '''We use units where J/k_B = 1.'''
    Tc = 2.0/np.log(1.0+np.sqrt(2.0))
    if T < Tc:
        return (1.0 - np.sinh(2.0/T)**(-4))**(1.0/8)
    else:
        return 0.0
magnetization_exact = np.vectorize(magnetization_exact_)

Tc = 2.0/np.log(1.0+np.sqrt(2.0))
T = np.linspace(0.01,3*Tc,1000)

In [ ]:
plt.plot(1.0/x,mx, 'o', ms=4, mfc='None', label = 'Mean Field')
plt.plot(T/Tc,magnetization_exact(T),'-', linewidth=1, label='Onsager', color=colors[0])

plt.xlabel(r'Temperature  $T/T_{\rm c}$')
plt.ylabel(r'Magnetization  $\langle m \rangle$')
plt.ylim(-0.001,1.05);
plt.xlim(0,2)
plt.legend();